This is for Sciencedirect

In [144]:
import time
from selenium import webdriver
import re
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import requests
import os
import shutil

# Set up the Chrome driver (you need to have Chrome installed)
driver = webdriver.Chrome()

# List of URLs to process
urls = [
    "https://www.sciencedirect.com/science/article/pii/S026382310800013X?via%3Dihub",
    "https://www.sciencedirect.com/science/article/pii/S0141029617304996#t0025",
    "https://www.sciencedirect.com/science/article/pii/S026382311730215X#t0010",
    "https://www.sciencedirect.com/science/article/pii/S026382311930196X#tbl2",
    "https://www.sciencedirect.com/science/article/pii/S0013794423006653#t0015"
]

# Directory to save the scraped files
directory = r"C:\Users\23109"

# Prefix to search for in file names
file_name_prefix = "Experiments on cold-formed steel columns with holes"

# Create the "scrape" folder if it doesn't exist
folder_path = "scrape"
os.makedirs(folder_path, exist_ok=True)

# Iterate over the URLs
cnt = 0
for url in urls:
    # Open the webpage
    driver.get(url)
    cnt += 1
    if cnt==1:
        # Wait for 20 seconds for manual login
        time.sleep(40)
    time.sleep(5)
    # Check if the button exists on the page
    button = driver.find_elements("xpath", "//span[contains(@class, 'button-link-text') and contains(text(), 'Show more figures')]")

    if button:
        # Click the button to load more figures
        button[0].click()
        time.sleep(5)  # Wait for the figures to load
    else:
        print("The 'Show more figures' button was not found.")

    # Retrieve the HTML content
    html_content = driver.page_source

    # Find all possible HTMLs with a similar format
    possible_htmls = re.findall(r'<div class="tables[^>]*>.*?</div>', html_content, flags=re.DOTALL)

    # Extract displayed text and save the tables as text and original HTML files
    for i, html_snippet in enumerate(possible_htmls):
        modified_html = html_snippet.replace('<span class="label">', '<span class="label" style="color: black;">')
        table_html = f"<div style='color:black;'>{modified_html}</div>"
        display(HTML(table_html))

        # Save the original HTML as a text file
        with open(f"table_{i+1}_original.html", "w", encoding="utf-8") as file:
            file.write(table_html)

        print(f"Table {i+1} saved as text and original HTML files.")

    # Download and save the images
    figure_links = re.findall(r'<img alt=".*?" class="u-display-block" height=".*?" src="(.*?)" width=".*?">', html_content)
    print('Number of figures:', len(figure_links))
    figure_links = [url.replace('.sml', '.jpg') for url in figure_links]

    for i, image_url in enumerate(figure_links):
        try:
            # Send a GET request to the image URL
            response = requests.get(image_url)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Get the file name from the URL
                file_name = f"{file_name_prefix}_{i+1}.jpg"

                # Save the image in the current working directory
                with open(file_name, "wb") as file:
                    file.write(response.content)

                print(f"Image {i+1} downloaded successfully!")
            else:
                print(f"Failed to download Image {i+1}.")
        except Exception as e:
            print(f"Error downloading Image {i+1}: {e}")

    # Move the files to the "scrape" folder
    for filename in os.listdir(directory):
        if filename.startswith(file_name_prefix):
            # Get the full path of the file
            file_path = os.path.join(directory, filename)

            # Move the file to the "scrape" folder
            destination_path = os.path.join(folder_path, filename)
            shutil.move(file_path, destination_path)

            print(f"File '{filename}' moved to 'scrape' folder.") 
            
    import re
    from IPython.display import display, HTML
    from bs4 import BeautifulSoup
    import pandas as pd

    # Find all possible HTMLs with a similar format
    # Create an empty DataFrame to store the merged tables
    merged_table = pd.DataFrame()

    # Iterate over the tables
    first_column_names = []

    # Iterate over the tables
    for html_snippet in possible_htmls:
        modified_html = html_snippet.replace('<span class="label">', '<span class="label" style="color: black;">')
        table_html = f"<div style='color:black;'>{modified_html}</div>"

        # Convert the HTML table to a DataFrame using BeautifulSoup
        soup = BeautifulSoup(table_html, 'html.parser')
        table = soup.find('table')
        headers = table.findAll('th')

        # Get the name of the first column
        if headers:
            first_column_name = headers[0].text.strip()
            first_column_names.append(first_column_name)

    # Find the most common string in the first column names
    most_common_column_name = max(set(first_column_names), key=first_column_names.count)

    # Iterate over the tables
    for i, html_snippet in enumerate(possible_htmls):
        modified_html = html_snippet.replace('<span class="label">', '<span class="label" style="color: black;">')
        table_html = f"<div style='color:black;'>{modified_html}</div>"

        # Convert the HTML table to a DataFrame using BeautifulSoup
        soup = BeautifulSoup(table_html, 'html.parser')
        table = soup.find('table')
        df = pd.read_html(str(table))[0]

        # Check if the first column contains the most_common_column_name
        if not any(most_common_column_name.lower() in str(col).lower() for col in df.columns):
            # Skip the table and report the issue
            print(f"Skipping Table {i+1} due to missing first column '{most_common_column_name}'")
            continue

        # Check if the first column contains the string "section" (case insensitive)
        section_columns = [col for col in df.columns if 'section' in str(col).lower()]
        if section_columns and section_columns[0] == df.columns[0]:
            # Drop the first column if it contains "section"
            df = df.drop(df.columns[0], axis=1)

        # Check if the DataFrame contains a column with the name containing "Specimen"
        specimen_columns = [col for col in df.columns if most_common_column_name in col]
        if len(specimen_columns) > 0:
            # Keep only one column with the name containing "Specimen"
            df = df.drop(specimen_columns[1:], axis=1)

        # Add a suffix to the columns to avoid overlap
        suffix = f"_{i+1}"
        df = df.add_suffix(suffix)

        # Merge the current table with the existing merged table
        if merged_table.empty:
            merged_table = df
        else:
            merged_table.columns = merged_table.columns.get_level_values(0)
            df.columns = df.columns.get_level_values(0)
            merged_table = pd.merge(merged_table, df, left_index=True, right_index=True, how='outer')

        # Save the original HTML as a text file
        with open(f"table_{i+1}_original.html", "w", encoding="utf-8") as file:
            file.write(table_html)
        #display(HTML(table_html))
        print(f"Table {i+1} saved as text and original HTML files.")

    # Get the column names dynamically that contain the most_common_column_name
    columns_to_merge = [col for col in merged_table.columns if most_common_column_name in col]

    # Merge the columns together and change the column name dynamically
    merged_table[most_common_column_name] = merged_table[columns_to_merge].fillna('').apply(lambda row: ''.join(row), axis=1)

    # Drop the redundant columns except for the merged column
    merged_table.drop(columns=columns_to_merge[1:], inplace=True)

    # Rename the column to "Specimen"
    merged_table = merged_table.rename(columns={most_common_column_name: "Specimen"})

    # Print the modified DataFrame
    print('')
    print('')
    print('Merged Table of Article ', cnt)
    display(merged_table)
    print('')
    print('')

    filename = f"Merged Table of article {cnt}.csv"
    merged_table.to_csv(filename, index=False)
    print(f'file{cnt}_saved!')
    print('')
    print('')

The 'Show more figures' button was not found.


Table 1 saved as text and original HTML files.


Table 2 saved as text and original HTML files.


Table 3 saved as text and original HTML files.


Table 4 saved as text and original HTML files.


...

Table 3 saved as text and original HTML files.


Specimen,Computed crack length (mm),Measured crack length (mm),Relative error (%)
625CSCT2-1,9.37,9.78,−4.2
625CSCT2-2,11.25,11.14,0.9
625CSCT2-3,12.25,11.59,5.6


Table 4 saved as text and original HTML files.
Number of figures: 6
Image 1 downloaded successfully!
Image 2 downloaded successfully!
Image 3 downloaded successfully!
Image 4 downloaded successfully!
Image 5 downloaded successfully!
Image 6 downloaded successfully!
File 'Experiments on cold-formed steel columns with holes_1.jpg' moved to 'scrape' folder.
File 'Experiments on cold-formed steel columns with holes_2.jpg' moved to 'scrape' folder.
File 'Experiments on cold-formed steel columns with holes_3.jpg' moved to 'scrape' folder.
File 'Experiments on cold-formed steel columns with holes_4.jpg' moved to 'scrape' folder.
File 'Experiments on cold-formed steel columns with holes_5.jpg' moved to 'scrape' folder.
File 'Experiments on cold-formed steel columns with holes_6.jpg' moved to 'scrape' folder.
Table 1 saved as text and original HTML files.
Table 2 saved as text and original HTML files.
Table 3 saved as text and original HTML files.
Skipping Table 4 due to missing first column 'M

,Material_1,Fe_1,Ni_1,Cr_1,Mo_1,Si_1,Mn_1,Al_1,Ti_1,Co_1,...,A (MPa-nh−1)_2,n_2,Designation_3,B (mm)_3,W (mm)_3,a0/W_3,K (MPam1/2)_3,Failure time t (h)_3,Remark_3,Specimen
0,IN625,4.19,61.10,21.55,9.5,0.23,0.337,0.22,0.315,–,...,9.500000e-35,11.6,625CT10-1,10,20,0.50,54.6,550,Fractured,IN625IN625IN625
1,IN783,24.88,28.21,3.24,–,0.06,–,5.32,0.320,34.29,...,1.340000e-46,15.2,625CT10-2,10,20,0.50,54.6,690,Fractured,IN783IN783IN625
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,625CT10-3,10,20,0.50,41.0,3590,Fractured,IN625
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,625CT10-4,10,20,0.50,27.3,10400,Interrupted,IN625
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,783CT10-1,10,20,0.51,35.2,140,Fractured,IN783
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,783CT10-2,10,20,0.51,30.2,518,Fractured,IN783
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,783CT10-3,10,20,0.50,24.6,>4580,Interrupted,IN783
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,625CSCT2-1,2,15,0.55,55.3,3,Fractured,IN625
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,625CSCT2-2,2,15,0.55,46.1,168,Fractured,IN625
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,625CSCT2-3,2,15,0.50,41.0,563,Fractured,IN625




file5_saved!




This is for ASCE

In [122]:
from selenium import webdriver
from IPython.display import Image, display
from bs4 import BeautifulSoup
import requests
import os
import shutil
import pandas as pd
import re

# List of URLs to process
urls = [
    'https://ascelibrary.org/doi/full/10.1061/%28ASCE%29ST.1943-541X.0000120',
    'https://ascelibrary.org/doi/full/10.1061/%28ASCE%29ST.1943-541X.0000310',
    'https://ascelibrary.org/doi/full/10.1061/%28ASCE%29ST.1943-541X.0002615',
    'https://ascelibrary.org/doi/full/10.1061/%28ASCE%29ST.1943-541X.0001265?mi=3i1ciu',
    'https://ascelibrary.org/doi/full/10.1061/%28ASCE%290733-9445%282005%29131%3A3%28457%29',
    'https://ascelibrary.org/doi/full/10.1061/%28ASCE%290733-9445%282006%29132%3A2%28212%29'
]

# Specify the directory to save the scraped files
directory = 'scrape'

# Create the "scrape" folder if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# List to store merged tables with "Specimen" column
merged_tables = []

# Loop over the URLs
for url in urls:
    # Set up the Chrome driver
    driver = webdriver.Chrome()
    
    # Open the webpage
    driver.get(url)
    
    # Retrieve the HTML content of the page
    html = driver.page_source
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all table elements on the page
    tables = soup.find_all('table')
    
    # Extract the table data and store it in a list of DataFrames
    dataframes = []
    for table in tables:
        headers = []
        data_rows = []
        
        # Find the table rows
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all(['td', 'th'])
            row_data = [cell.text.strip() for cell in cells]
            data_rows.append(row_data)
            
            # Update the number of headers if necessary
            if len(row_data) > len(headers):
                headers = row_data
        
        # Create a DataFrame from the table data
        df = pd.DataFrame(data_rows, columns=headers)
        
        # Check if the DataFrame contains a column with "Specimen" substring
        specimen_columns = [col for col in df.columns if 'specimen' in col.lower()]
        if specimen_columns:
            dataframes.append(df)
            display(df)
        
    # Merge the tables with "Specimen" column
    if dataframes:
        # Append a suffix to duplicate column names
        for i in range(len(dataframes)):
            if dataframes[i].columns.duplicated().any():
                suffixes = ['_' + str(j) for j in range(dataframes[i].shape[1])]
                dataframes[i].columns = [f'{col}{suffixes[j]}' for j, col in enumerate(dataframes[i].columns)]
        
        merged_table = pd.concat(dataframes, ignore_index=True)
        merged_tables.append(merged_table)
    
    # Save the DataFrames as separate CSV files
    for i, df in enumerate(dataframes):
        filename = re.sub(r'\W+', '_', url) + f'_table_{i+1}.csv'
        filepath = os.path.join(directory, filename)
        df.to_csv(filepath, index=False)
    
    # Close the Chrome driver
    driver.quit()

# Merge all the tables with "Specimen" column
if merged_tables:
    final_merged_table = pd.concat(merged_tables, ignore_index=True)
    contains_specimen = [bool('specimen' in col.lower()) for col in final_merged_table.columns]
    
    merge_columns = final_merged_table.columns[contains_specimen].tolist()

    # Create a new DataFrame to store the merged result
    merged_table = final_merged_table.copy()

    # Merge the columns into a single column called "Specimen"
    merged_table['Specimen'] = merged_table[merge_columns].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

    # Drop the columns from merge_columns list except for "Specimen"
    columns_to_drop = [col for col in merge_columns if col != "Specimen"]
    merged_table.drop(columns=columns_to_drop, inplace=True)
    if "Specimen" in merged_table.columns:
        # Move the "Specimen" column to the first position
        columns = list(merged_table.columns)
        columns.remove("Specimen")
        merged_table = merged_table[["Specimen"] + columns]

    # Display the modified table
    display(merged_table)    
    
    
    print("This is the merged table.")
else:
    print("No tables with 'Specimen' column found.")

,Specimen (n=10),H (mm),B (mm),Bl (mm),t (mm),L (mm),E (GPa),fy (MPa),fu (MPa),εf (%)
0,Specimen (n=10),H (mm),B (mm),Bl (mm),t (mm),L (mm),E (GPa),fy (MPa),fu (MPa),εf (%)
1,P-T5-minor-A,100.25,49.96,—,4.46,990,64.7,167.9,198.9,7.0
2,P-T5-minor-A-R,100.08,49.60,—,4.44,990,64.7,167.9,198.9,7.0
3,P-T5-minor-B,86.90,48.01,—,1.90,"3,040",66.8,181.3,200.6,6.9
4,P-T6-minor-B,86.78,48.01,—,1.86,"3,040",65.0,226.6,252.7,6.3
5,L-T5-minor-C,79.91,37.04,13.41,1.46,"3,040",62.5,175.2,186.1,7.2
6,L-T6-minor-C,80.15,37.01,13.47,1.49,"3,040",62.2,238.5,249.9,7.4
7,P-T5-major-B,86.79,49.96,—,1.93,"3,040",66.8,181.3,200.6,6.9
8,P-T6-major-B,86.81,49.97,—,1.95,"3,040",65.0,226.6,252.7,6.3
9,L-T5-major-C,80.09,36.99,13.42,1.46,"3,040",62.5,175.2,186.1,7.2


,Specimen (n=10),MEXP (kN·m),MFE (kN·m),MEXPMFE
0,Specimen (n=10),MEXP (kN·m),MFE (kN·m),MEXPMFE
1,P-T5-minor-A,1.92,1.91,1.01
2,P-T5-minor-A-R,1.94,1.90,1.02
3,P-T5-minor-B,0.39,0.38,1.03
4,P-T6-minor-B,0.44,0.42,1.05
5,L-T5-minor-C,0.57,0.55,1.04
6,L-T6-minor-C,0.78,0.72,1.08
7,P-T5-major-B,2.50,2.72,0.92
8,P-T6-major-B,3.17,3.14,1.01
9,L-T5-major-C,2.59,2.65,0.98


,Specimen (Test: 6; FE: 25),Mu (kN·m),MuMAA,MuMAS/NZS,MuMEC9,MuMCN,MuMCSM,MuMDSM,MuMDSM−1
0,Specimen (Test: 6; FE: 25),Mu (kN·m),MuMAA,MuMAS/NZS,MuMEC9,MuMCN,MuMCSM,MuMDSM,MuMDSM−1
1,P-T5-minor-A,1.92,1.36,1.96,1.37,1.92,1.24,0.97,0.97
2,P-T5-minor-A-R,1.94,1.38,2.02,1.41,1.97,1.27,1.00,1.00
3,P-T5-M1-minor,1.27,0.92,1.68,1.36,1.31,1.21,1.48,1.48
4,P-T5-M2-minor,1.39,0.90,1.77,1.40,1.34,1.22,1.45,1.45
5,P-T5-M3-minor,0.89,0.98,1.56,1.39,1.33,1.22,1.37,1.37
6,P-T5-M4-minor,0.89,1.27,1.75,1.38,1.32,1.16,1.32,1.32
7,P-T5-M5-minor,0.89,1.44,1.96,1.37,1.31,1.20,1.36,1.29
8,P-T5-minor-B,0.39,1.09,1.50,0.99,1.00,1.37,0.94,0.94
9,P-T5-M1-minor,1.10,1.19,1.47,1.18,1.18,1.01,1.19,1.19


,Specimen (Test: 4; FE: 20),Mu (kN·m),MuMAA,MuMAS/NZS,MuMEC9,MuMCN,MuMCSM,MuMDSM,MuMDSM−1
0,Specimen (Test: 4; FE: 20),Mu (kN·m),MuMAA,MuMAS/NZS,MuMEC9,MuMCN,MuMCSM,MuMDSM,MuMDSM−1
1,P-T5-major-B,1.25,0.82,0.92,3.81,2.62,0.94,1.03,1.00
2,P-T5-M1-major,1.79,1.13,1.15,4.51,3.09,0.95,0.89,1.11
3,P-T5-M2-major,4.21,0.87,1.01,4.30,3.27,1.14,0.88,1.03
4,P-T5-M3-major,2.44,1.13,1.05,4.16,2.84,1.08,0.97,0.98
5,P-T5-M4-major,3.49,1.19,1.07,3.96,2.82,1.10,0.96,1.37
6,P-T5-M5-major,4.46,1.23,1.21,3.64,2.67,1.05,0.91,1.33
7,P-T6-major-B,1.59,0.93,0.99,3.98,2.66,1.05,0.92,1.25
8,P-T6-M1-major,1.94,1.37,1.05,4.10,2.68,0.89,0.83,1.10
9,P-T6-M2-major,5.15,0.95,1.04,4.38,3.20,1.22,0.92,1.12


,Specimen,PTest(kN),eTest(mm),Failure mode,PFE(kN),eFE(mm),Failure mode,PTestPFE,eTesteFE
0,,Test,FE,Test/ FE,None,None,None,None,None
1,Specimen,PTest(kN),eTest(mm),Failure mode,PFE(kN),eFE(mm),Failure mode,PTestPFE,eTesteFE
2,P1.2L250,23.8,0.54,L,24.9,0.61,L,0.96,0.89
3,P1.2L1000,18.7,1.10,F+FT,18.1,0.96,F+FT,1.03,1.14
4,P1.2L1500,15.2,0.82,F+FT,15.7,0.70,F+FT,0.97,1.17
5,P1.2L2000,12.6,1.66,F+FT,11.7,1.74,F+FT,1.08,0.95
6,P1.2L2500,11.6,1.25,F+FT,10.1,1.04,F+FT,1.15,1.20
7,P1.2L3000,8.0,1.07,F+FT,8.6,0.98,F,0.93,1.15
8,P1.2L3500,5.8,1.03,F+FT,6.4,0.89,F,0.91,1.16
9,Mean,—,—,—,—,—,—,1.00,1.09


,Specimen,bt,PFE(kN),eFE(mm),Failure mode
0,,,FE analysis,None,None
1,Specimen,bt,PFE(kN),eFE(mm),Failure mode
2,P0.8L250,85,9.6,0.23,L
3,P0.8L1000,85,8.2,0.47,L+FT
4,P0.8L1500,85,6.6,0.57,L+FT
5,P0.8L2000,85,5.3,0.65,F
6,P0.8L2500,85,4.3,0.70,F
7,P0.8L3000,85,3.8,0.62,F
8,P0.8L3500,85,3.5,0.65,F
9,P1.0L250,65,15.6,0.22,L


,Specimen,H (mm),B (mm),Bl (mm),t (mm),L (mm),E (GPa),fy (MPa),fu (MPa),εf (%),...,Failure mode_3,PFE(kN)_4,eFE(mm)_5,Failure mode_6,PTestPFE_7,eTesteFE_8,bt,PFE(kN),eFE(mm),Failure mode
0,Specimen (n=10),H (mm),B (mm),Bl (mm),t (mm),L (mm),E (GPa),fy (MPa),fu (MPa),εf (%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P-T5-minor-A,100.25,49.96,—,4.46,990,64.7,167.9,198.9,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P-T5-minor-A-R,100.08,49.60,—,4.44,990,64.7,167.9,198.9,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P-T5-minor-B,86.90,48.01,—,1.90,"3,040",66.8,181.3,200.6,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P-T6-minor-B,86.78,48.01,—,1.86,"3,040",65.0,226.6,252.7,6.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,P10.0L1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,297.7,1.88,F
153,P10.0L2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,232.6,2.00,F
154,P10.0L2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,170.9,2.05,F
155,P10.0L3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,125.9,1.97,F


This is the merged table.
